In [ ]:
from gym.spaces import Box
import gym
import os
import numpy as np
import cv2
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.utils import set_random_seed

# 1. DEFINIZIONE DELLE CARTELLE
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

class MarioModernized(gym.Env):
    def __init__(self):
        super().__init__()
        # Creiamo l'ambiente originale
        inner_env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
        self.mario = JoypadSpace(inner_env, SIMPLE_MOVEMENT)
        
        # IA vede 84x84 in scala di grigi
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = self.mario.action_space

    def _process_frame(self, frame):
        if frame is not None:
            # Scala di grigi
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            # Resize 84x84
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            
            # --- AGGIUNTA: VISUALIZZAZIONE "OCCHI IA" ---
            # Ingrandiamo l'immagine per renderla visibile a noi umani (es. 256x256)
            view_img = cv2.resize(resized, (256, 256), interpolation=cv2.INTER_NEAREST)
            cv2.imshow("Cosa vede l'IA (Grigio 84x84)", view_img)
            cv2.waitKey(1) # Necessario per aggiornare la finestra di OpenCV
            # --------------------------------------------
            
            return resized[:, :, np.newaxis]
        return np.zeros((84, 84, 1), dtype=np.uint8)

    def reset(self, seed=None, options=None):
        # Il vecchio gym-mario non supporta il seed nel reset, lo gestiamo a monte
        obs = self.mario.reset()
        self.mario.render() # Finestra di gioco originale
        obs = self._process_frame(obs)
        return obs 

    def step(self, action):
        obs, reward, done, info = self.mario.step(action)
        self.mario.render()
        obs = self._process_frame(obs)
        return obs, float(reward), done, info

    def render(self, mode='human'):
        return self.mario.render()

if __name__ == "__main__":
    # 2. SETTAGGIO SEED (42)
    # Imposta il seed per numpy, random e pytorch per SB3
    SEED = 42
    set_random_seed(SEED)

    # Creazione ambiente vettorializzato
    # Passiamo il seed anche alla creazione dell'ambiente
    env = DummyVecEnv([lambda: MarioModernized()])
    env.seed(SEED)
    
    # FrameStack di 4 frame
    env = VecFrameStack(env, n_stack=4, channels_order='last')

    # 3. CONFIGURAZIONE DEL CALLBACK
    checkpoint_callback = CheckpointCallback(
        save_freq=100000, 
        save_path=CHECKPOINT_DIR,
        name_prefix='mario_model'
    )

    # 4. CONFIGURAZIONE PPO CON SEED
    model = PPO(
        "CnnPolicy", 
        env, 
        verbose=1, 
        learning_rate=0.0001,
        n_steps=512,
        batch_size=64,
        ent_coef=0.01,
        tensorboard_log=LOG_DIR,
        seed=SEED # <--- Seed impostato nel modello
    )

    print(f"Training avviato con SEED: {SEED}")
    print(f"Visualizzazione 'Occhi IA' attiva via OpenCV.")
    
    try:
        model.learn(total_timesteps=1000000, callback=checkpoint_callback)
    except KeyboardInterrupt:
        print("Salvataggio finale in corso...")
        model.save("mario_model_final")
        cv2.destroyAllWindows() # Chiude le finestre di OpenCV alla fine